Aqui montaremos a rede para avaliar os dados para a tarefa de classificação multiclasse com a extração de características hog (Histogram of Oriented Gradients)

1º precisamos importar as bibliotecas necessárias e carregar os dados do conjunto MNIST 

In [1]:
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Função para extrair as características HOG

In [2]:
def extract_hog_features(images):
    features = []
    for image in images:
        fd = hog(image, orientations=8, pixels_per_cell=(4, 4),
                 cells_per_block=(1, 1), visualize=False)
        features.append(fd)
    return np.array(features)

Precisamos pré-processar os dados, isto é, normalizá-los

In [3]:
# Pré-processamento: Conversão para escala de cinza e normalização
#x_train = np.array([color.rgb2gray(i) for i in x_train])
#x_test = np.array([color.rgb2gray(i) for i in x_test])

# COMENTEI O CÓDGIO ACIMA POIS AS IMAGENS JÁ VEM NA ESCALA DE CINZA E EM ARRAYS DO NUMPY

x_train = x_train / 255.0
x_test = x_test / 255.0

Agora, extraíremos as características HOG do conjunto

In [4]:
# Extrair características HOG
x_train_hog = extract_hog_features(x_train)
x_test_hog = extract_hog_features(x_test)

E prepararemos os dados para o treinamento

In [5]:
# 4. Preparar os dados para treinamento
# Normalizar as características HOG
scaler = StandardScaler()
x_train_hog = scaler.fit_transform(x_train_hog)
x_test_hog = scaler.transform(x_test_hog)

# Redimensionar para o formato esperado pela CNN
x_train_hog = x_train_hog.reshape(x_train_hog.shape[0], 1, 1, x_train_hog.shape[1])
x_test_hog = x_test_hog.reshape(x_test_hog.shape[0], 1, 1, x_test_hog.shape[1])

# Converter rótulos para one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

3º Agora, criaremos o modelo da CNN que aceita características HOG para a tarefa multiclasse e compilaremos ele

In [6]:
model = Sequential([
    Conv2D(32, kernel_size=(1, 1), activation='relu', input_shape=(1, 1, x_train_hog.shape[3])),
    MaxPooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
# 6. Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\mtoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-hog/hiperparametros.json

- Salvando os pesos iniciais

In [7]:
# Definindo os hiperparâmetros
hiperparametros = {
    "arquitetura": {
        "camadas": [64, 64, 10],
        "ativacao": "relu",
        "ultimo_ativacao": "softmax"
    },
    "inicializacao": {
        "pesos": "he_normal",
        "bias": "zeros"
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-hog/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-hog/pesos_iniciais.weights.h5')

4º Treinar o modelo

In [8]:
# 7. Treinar o modelo
historico = model.fit(x_train_hog, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8623 - loss: 0.4596 - val_accuracy: 0.9675 - val_loss: 0.1026
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9764 - loss: 0.0729 - val_accuracy: 0.9676 - val_loss: 0.1066
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9831 - loss: 0.0532 - val_accuracy: 0.9712 - val_loss: 0.0979
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9871 - loss: 0.0381 - val_accuracy: 0.9722 - val_loss: 0.1083
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9900 - loss: 0.0306 - val_accuracy: 0.9687 - val_loss: 0.1261
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9910 - loss: 0.0263 - val_accuracy: 0.9733 - val_loss: 0.1184
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9941 - loss: 0.0184 - val_accuracy: 0.9740 - val_loss: 0.1284
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9942 - loss: 0.0169 - 

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [9]:
# PESOS FINAIS
model.save_weights('arquivos-hog/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-hog/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_train_hog)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-hog/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


5º Testar o modelo

In [10]:
# 8. Avaliar o modelo
test_loss, test_acc = model.evaluate(x_test_hog, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.9676 - loss: 0.1599
Test accuracy: 0.9731000065803528, Test loss: 0.13391050696372986
